## Stream video and subtract background model and track rodent position
This program calculates pixel distance away from two objects and time spent with these objects

Controls for background model estimation:
1. Background model can be reestimated by pressing space bar. It will continue to use the video to estimate the background model unitl space bar is pressed again. 
2. +|- keys can be pressed to further filter out white blobs in the foreground streaming video.
3. 's' can be pressend to toggle shadows

Cell Functionality:
1. Imports
2. Find homography
3. Find pixel locations of object and set exploration classification radius
4. Setup background model if not estimated in 2. 
5. Convert video file so it is compressed and easier to process
6. Chop off part of the video
7. Position tracking

In [1]:
#Imports
import numpy as np
from skimage.measure import label, regionprops, find_contours
import imageio
import cv2
import operator
import math
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
#Find homography
source_pts = np.array([[100, 90], [475, 70], [425, 370], [75, 470]], dtype = 'float32') #pixels!
dst_pts = np.array([[0, 0], [100, 0], [80, 80], [0, 100]], dtype = 'float32') #cm

homography_matrix, _ = cv2.findHomography(source_pts, dst_pts, cv2.RANSAC, 5.0) #not sure what the 5.0 is really doing seems to work though
POINTMOO = np.array([[75,470]],dtype = 'float32')
POINTMOO = np.array([POINTMOO])
dst_pt = cv2.perspectiveTransform(POINTMOO, homography_matrix)
#print(dst_pt[0][0][0])
#print(dst_pt[0][0][1])
#print(dst_pt)

In [3]:
 #Locations of objects and distance thresholds (how close does the rat have to be to be near it)
familiarObject_center_x = 380
familiarObject_center_y = 160
realWorld_famObj_center = cv2.perspectiveTransform(np.array([np.array([[familiarObject_center_x,familiarObject_center_y]], dtype='float32')]), homography_matrix)
familiarObject_center_x_realWorld = realWorld_famObj_center[0][0][0]
familiarObject_center_y_realWorld = realWorld_famObj_center[0][0][1]
DistanceThreshold_familiarObject = 22;
numFrames_FamiliarObject = 0;

novelObject_center_x = 210
novelObject_center_y = 320
realWorld_novelObj_center = cv2.perspectiveTransform(np.array([np.array([[novelObject_center_x,novelObject_center_y]],dtype='float32')]), homography_matrix)
novelObject_center_x_realWorld = realWorld_novelObj_center[0][0][0]
novelObject_center_y_realWorld = realWorld_novelObj_center[0][0][1]
DistanceThrehshold_NovelObject = 22;
numFrames_NovelObject = 0;

In [4]:
#setup for background model and foreground tracking
if 'fgbg' not in locals():
    fgbg = cv2.createBackgroundSubtractorKNN()
    
morph_size = 2
shadowValue = 127
learnBG = False
showShadow = False

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(morph_size,morph_size))

# Convert video

In [5]:
 '''
## convert a video file to an .mp4 using imageio (ffmpeg) and default settings---usually much smaller file size

import imageio
import os
from tqdm import tqdm_notebook

infile = 'tornado_125mg_rot_10_6_16_pt2.mov'

verbose = False

try:
    reader = imageio.get_reader(infile)
    fps = reader.get_meta_data()['fps']
    
    if verbose:
        print('input video file has a framerate of {} fps'.format(fps))
    
    try:
        writer = imageio.get_writer('converted.mp4', fps=fps,)
        for im in tqdm_notebook(reader, desc='converting video'):
            writer.append_data(im)

        writer.close()
        print('conversion complete!')
    
    except:
        print("something went wrong!")
except:
    print("something went wrong; couldn't open file?...")
    
# TODO: rename files (unless explicitly told not to)
splits = infile.split('.')
newName = splits[0] + '-orig.' + splits[1]
os.rename(infile, newName)
os.rename('converted.mp4', infile)
'''

'\n## convert a video file to an .mp4 using imageio (ffmpeg) and default settings---usually much smaller file size\n\nimport imageio\nimport os\nfrom tqdm import tqdm_notebook\n\ninfile = \'tornado_125mg_rot_10_6_16_pt2.mov\'\n\nverbose = False\n\ntry:\n   reader = imageio.get_reader(infile)\n   fps = reader.get_meta_data()[\'fps\']\n   \n   if verbose:\n       print(\'input video file has a framerate of {} fps\'.format(fps))\n   \n   try:\n       writer = imageio.get_writer(\'converted.mp4\', fps=fps,)\n       for im in tqdm_notebook(reader, desc=\'converting video\'):\n           writer.append_data(im)\n\n       writer.close()\n       print(\'conversion complete!\')\n   \n   except:\n       print("something went wrong!")\nexcept:\n   print("something went wrong; couldn\'t open file?...")\n   \n# TODO: rename files (unless explicitly told not to)\nsplits = infile.split(\'.\')\nnewName = splits[0] + \'-orig.\' + splits[1]\nos.rename(infile, newName)\nos.rename(\'converted.mp4\', infile

In [6]:
"""
#chop off first X seconds
import subprocess
seconds = "22"
subprocess.call(['ffmpeg','-i', 'Round2/Day3/Test-2-cropped.mkv', '-ss', seconds, 'Round2/Day3/Test-2-cropped1.mkv'])
"""

'\n#chop off first X seconds\nimport subprocess\nseconds = "22"\nsubprocess.call([\'ffmpeg\',\'-i\', \'Round2/Day3/Test-2-cropped.mkv\', \'-ss\', seconds, \'Round2/Day3/Test-2-cropped1.mkv\'])\n'

# Actual work happens below

In [ ]:
#File IO
numFrames_FamiliarObject = 0
numFrames_NovelObject = 0
reader = imageio.get_reader('Round2/Day3/Test-2-cropped1.mkv')
fps = reader.get_meta_data()['fps']
print('input video file length is {} seconds'.format(reader.get_length()/(fps)))
print('input video file has a framerate of {} fps'.format(fps))
writer = imageio.get_writer('test-out.mp4', fps=fps)

#Read in file frame by frame. Perform position tracking background subtraction
# cv2.morph_open
for i, im in enumerate(reader):
    im =  cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
    #im = im[10:470, 20:480]
    if learnBG:
        fgmask = fgbg.apply(im)
    else:
        fgmask = fgbg.apply(im, learningRate=0)
    
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(8*morph_size,8*morph_size)))
    bg = fgbg.getBackgroundImage()
    
    # see https://www.mathworks.com/matlabcentral/answers/68696-how-can-i-extract-the-largest-blob-in-a-binary-image
    label_img = label(fgmask)
    regions = regionprops(label_img)
    
    region_areas = []
    
    for props in regions:
        region_areas.append(props.area)
    
    if len(region_areas) > 0:
        largestBlobIndex, _ = max(enumerate(region_areas), key=operator.itemgetter(1))
    
        ratBlob = regions[largestBlobIndex]
        
        #print(ratBlob.perimeter)
        #ratContours = find_contours(fgmask,0.8)
        #print(ratContours)
        #ratContours = np.asarray(ratContours).reshape(-1,1,2).astype(np.int32)
        #print(ratContours)
        #cv2.drawContours(im, ratContours,0,(0,255,0),2)
        #cv2.putText(im, str(ratContours[0][0]), (30,30), cv2.FONT_HERSHEY_PLAIN,2,255)
        
        y0, x0 = ratBlob.centroid
        
#         xval = []
#         yval = []
        data = ratBlob.coords
        moo = data.reshape(data.size)
        xval = moo[::2]
        yval = moo[1::2]
#         for point in np.nditer(data):
#             xval.append(point[0])
#             yval.append(point[1])
#         xarray = nd.array(xval)
#         yarray = nd.array(yval)
        coeffs = np.polyfit(np.array(xval), np.array(yval), 1)
#         line = coeffs[1]*xval + coeffs[2]
        largestx = max(xval)
        smallestx = min(xval)
        cv2.arrowedLine(im, (int(np.polyval(coeffs, int(smallestx))), int(smallestx)), (int(np.polyval(coeffs, int(largestx))), int(largestx)), (255,0,0), 2)
#         cv2.arrowedLine(im, (int(smallestx), int(np.polyval(coeffs, int(smallestx)))), (int(largestx), int(np.polyval(coeffs, int(largestx)))), (255,0,0), 2)
#         cv2.arrowedLine(im, (int(x0), int(y0)), (int(smallestx), int(np.polyval(coeffs, int(smallestx)))), (255,0,0), 2)
#         xFit = np.linspace(1, largestx, 10)
#         yFit = np.polyval(coeffs, xFit)
        
#         pltim = plt.imread('Round2/Day3/Test-2-cropped.mkv')
#         implot = plt.imshow(pltim)
    
#         plt.plot(xFit, yFit, 'b--')

#         plt.show()
    
#MOOO
        #Contour and line of best fit
#         _, contours, _ = cv2.findContours(fgmask,cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
#         #print(contours[0])
#         #print(ratContours)
#         rows, cols = im.shape[:2]
#         [vx,vy,x,y] = cv2.fitLine(contours[0], cv2.DIST_L2,0,0.01,0.01)
#         lefty = int((-x*vy/vx) + y)
#         righty = int(((cols-x)*vy/vx) + y)
#         cv2.drawContours(im, contours, 0,(0,255,0),2)
#         cv2.arrowedLine(im,(cols-1,righty),(0,lefty),(255,0,0),2)

        #draw tracking "dot"
        cv2.circle(im,(int(x0),int(y0)),10,(255,255,255),-11)
        cv2.circle(im,(int(x0),int(y0)),11,(0,0,255),1) # draw circle
        cv2.ellipse(im, (int(x0),int(y0)), (10,10), 0, 0, 90,(0,0,255),-1 )
        cv2.ellipse(im, (int(x0),int(y0)), (10,10), 0, 180, 270,(0,0,255),-1 )
        cv2.circle(im,(int(x0),int(y0)),1,(0,255,0),1) # draw center
        #cv2.putText(OriImage,pid,(int(cx)+10,int(cy)-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(255,180,180))
        
        # 'dot' location of familiar object
        cv2.circle(im,(familiarObject_center_x,familiarObject_center_y),1,(0,0,0),10) 
        
        # 'dot' location of novel object
        cv2.circle(im,(novelObject_center_x,novelObject_center_y),1,(0,0,0),10) 
        
        realWorldPoint = cv2.perspectiveTransform(np.array([np.array([[x0,y0]],dtype='float32')]), homography_matrix)
        realWorldX = realWorldPoint[0][0][0]
        realWorldY = realWorldPoint[0][0][1]
        
        distanceFromNovelObject = math.hypot(novelObject_center_x_realWorld - realWorldX, novelObject_center_y_realWorld - realWorldY)
        distanceFromFamiliarObject = math.hypot(familiarObject_center_x_realWorld - realWorldX, familiarObject_center_y_realWorld - realWorldY)
        if(distanceFromNovelObject < DistanceThrehshold_NovelObject):
            numFrames_NovelObject = numFrames_NovelObject + 1
            cv2.circle(im,(novelObject_center_x,novelObject_center_y),1,(0,255,0),10) 

        if(distanceFromFamiliarObject < DistanceThreshold_familiarObject):
            numFrames_FamiliarObject = numFrames_FamiliarObject + 1
            cv2.circle(im,(familiarObject_center_x,familiarObject_center_y),1,(0,255,0),10) 
            
        
    cv2.imshow('fgmask',fgmask)
    cv2.imshow('im',im)
    cv2.imshow('bg',bg)
    
    im =  cv2.cvtColor(im, cv2.COLOR_BGR2RGB) # imageio writer takes RGB

    writer.append_data(bg)
    
    k = cv2.waitKey(1) & 0xff
#    if k!= 255:
#        print(k)
    if k == 32: # 'space'
        if learnBG:
            learnBG = False
            print('background learning OFF')
        else:
            learnBG = True
            print('background learning ON')
    if k == 115: # 's'
        if showShadow:
            showShadow = False
            shadowValue = 0
            print('shadows OFF')
        else:
            showShadow = True
            shadowValue = 127
            print('shadows ON')
        #fgbg.setDetectShadows(showShadow)
        fgbg.setShadowValue(shadowValue)
            
    if k == 171 or k == 43: # '+'
        if morph_size < 20:
            morph_size +=5
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(morph_size,morph_size))
    if k == 173 or k == 45: # '-'
        if morph_size > 2:
            morph_size -=1
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(morph_size,morph_size))
    if k == 27:
        break
        
        
print("Total amount of time spent with objects: {} seconds".format((numFrames_FamiliarObject/fps)+(numFrames_NovelObject/fps)))
print("Percentage of time spent with objects that was spent with the novel object: {}%".format((numFrames_NovelObject*100/fps)/((numFrames_FamiliarObject/fps)+(numFrames_NovelObject/fps))))
writer.close()
cv2.destroyAllWindows()
print('exited gracefully')

timeSpentFamObject = numFrames_FamiliarObject/fps
timeSpentNovObject = numFrames_NovelObject/fps

timeSpent = ('Familiar', 'Novel')
n_groups = len(timeSpent)
index = np.arange(n_groups)
bar_width = 0.1

plt.bar(index, [timeSpentFamObject, timeSpentNovObject], bar_width, color='blue', align='center', alpha=0.6)

plt.title("Time Spent with Objects")
plt.xticks(index, ('Familiar', 'Novel'))
plt.xlabel("Object")
plt.ylabel("Time (Seconds)")
plt.show()

input video file length is 675.2333333333333 seconds
input video file has a framerate of 30.0 fps
background learning ON
shadows ON
shadows OFF
background learning OFF
background learning ON
background learning OFF
background learning ON
background learning OFF
background learning ON
background learning OFF


In [ ]:
cow = data[0:10]
cow.squeeze()
np.savez("cow.npz",ariel=cow)

In [ ]:
cow.size

In [ ]:
moo = cow.reshape(20)
moo[1::2]